In [1]:
import pandas as pd

# Load and prepare the data
data = pd.read_parquet('gwas_fm_hyp.parquet')

# Unique chromosomes and cs_99 values
chromosomes = data['#chrom'].unique()
cs_99_values = data['cs_99'].unique()

# List to collect records of causal=1 SNPs
causal_records = []

# Loop through each chromosome and cs_99 value
for chrom in chromosomes:
    for cs_99_value in cs_99_values:
        # Filter data for current chromosome and cs_99 value
        df = data[(data['#chrom'] == chrom) & (data['cs_99'] == cs_99_value)]
        df.reset_index(drop=True, inplace=True)
        
        # If no data, continue to next iteration
        if df.empty:
            continue
        
        # Rank SNPs by prob value within the current group
        df['rank'] = df['prob'].rank(ascending=False, method='min')
        
        # Further filter for causal=1 SNPs
        causal_df = df[df['causal'] == 1]
        for index, row in causal_df.iterrows():
            # Create a record for each causal=1 SNP
            record = {
                'chromosome': chrom,
                'cs_99': cs_99_value,
                'position': row['pos'],
                'rank': row['rank']
            }
            causal_records.append(record)

# Create a new DataFrame from causal_records
causal_rankings_df = pd.DataFrame(causal_records)

# Now causal_rankings_df holds the ranking of all causal=1 SNPs


C:\Users\Windows\AppData\Local\Temp\ipykernel_18948\3741004241.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rank'] = df['prob'].rank(ascending=False, method='min')
C:\Users\Windows\AppData\Local\Temp\ipykernel_18948\3741004241.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rank'] = df['prob'].rank(ascending=False, method='min')
C:\Users\Windows\AppData\Local\Temp\ipykernel_18948\3741004241.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [2]:
causal_rankings_df.sort_values(by='rank', ascending=True) 

,chromosome,cs_99,position,rank
10,11,1.0,100739815,1.0
16,17,2.0,63478937,32.0
6,10,2.0,113963919,45.0
1,2,1.0,26700700,49.0
15,17,2.0,63471587,115.0
0,2,1.0,26697157,160.0
4,4,1.0,155721732,172.0
9,10,2.0,114045297,292.0
8,10,2.0,114033028,318.0
5,4,2.0,155512156,324.0


In [3]:
causal_rankings_df.describe()

,chromosome,cs_99,position,rank
count,17.000000,17.000000,1.700000e+01,17.000000
mean,8.529412,2.000000,8.112643e+07,478.411765
std,4.963840,0.707107,6.147250e+07,682.665462
min,2.000000,1.000000,1.852002e+06,1.000000
25%,4.000000,2.000000,2.669716e+07,115.000000
50%,10.000000,2.000000,1.007398e+08,318.000000
75%,11.000000,2.000000,1.140453e+08,391.000000
max,17.000000,3.000000,1.617270e+08,2320.000000
